In [1]:
import numpy as np
import pandas as pd
import requests
import datetime
import time

In [2]:
# all stocks from Fidelity website
stock_list = pd.read_csv('screener_results.csv')
stock_list = stock_list.loc[:, ['Symbol', 'Sector', 'Industry']]

In [ ]:
# split_stock_list = np.array_split(stock_list, 5)

In [ ]:
symbol = 'MSFT'

print(f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol={symbol}&outputsize=full&apikey={Special_API_key}')

In [ ]:
# restore with 
%store -r error_load
%store -r error_date
%store -r master_df
%store -r x 

In [3]:
master_df = pd.DataFrame()

# counter (corresponds to stock_list index)
x = 0

error_load = pd.Series([], dtype=str)
error_date = pd.Series([], dtype=str)

In [4]:
from secrets import special_api_key

import functions as f

In [ ]:
def extract_single_set_of_financial_statements(income_data, cashflow_data, balancesheet_data, years_back):
    inc_data = income_data['annualReports'][-years_back].copy()
    cas_data = cashflow_data['annualReports'][-years_back].copy()
    bal_data = balancesheet_data['annualReports'][-years_back].copy()
    
    return inc_data, cas_data, bal_data

In [ ]:
inc_data, cas_data, bal_data = f.extract_single_set_of_financial_statements(income_data, cash_data, balance_data, 1)

In [93]:
error_load = pd.Series([], dtype=str)
error_date = pd.Series([], dtype=str)

In [94]:
one = pd.Series([[2, 3, 2, 1]])
two = pd.Series([[4, 2, 1, 6]])

In [98]:
error_load = error_load.append(one, ignore_index=True)
error_load = error_load.append(two, ignore_index=True)

/tmp/ipykernel_1207619/423028936.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_load = error_load.append(one, ignore_index=True)
/tmp/ipykernel_1207619/423028936.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  error_load = error_load.append(two, ignore_index=True)


In [100]:
error_load

0    [2, 3, 2, 1]
1    [4, 2, 1, 6]
2    [2, 3, 2, 1]
3    [4, 2, 1, 6]
dtype: object

In [ ]:
error_load = pd.Series([], dtype=str)

# Create List of list
for i in range(5):
    error_load.append([])
    for j in range(3): 
        list[i].append(j)

print(lst) 

In [11]:
def adds_symbol_to(error_list):    
    error_list = pd.concat([error_list, pd.Series(stock)])
    return error_list 

0    0    0    0    0    0
dtype: int64
0          ...
0                                                    4
dtype: object

In [5]:
# keep track of ranges (underlying facts about the data)
# comparing first_date which is 5 to 11 days after earnings is announced to 
# second_date which is 341 to 347 days after earnings is announced 

master_df = pd.DataFrame()

# counter (corresponds to stock_list index)
x = 0

error_load = pd.Series([], dtype=str)
error_date = pd.Series([], dtype=str)

for stock, sector, industry  in stock_list.values:
    
    income_data, cash_data, balance_data = f.make_api_call_for_financial_statements(stock, special_api_key)
    price_data = f.make_api_call_for_price_time_series(stock, special_api_key)
    
    for years_back in range(5):
        
        try: 

            inc_data, cas_data, bal_data = f.extract_single_set_of_financial_statements(income_data, cash_data, balance_data, years_back)

        except KeyError: 
            # len() should return 3 for each one, should return 0 if a valid api call was not performed, should return 1 if api call limit has been reached
            print(f"Error: {len(income_data)}{len(cash_data)}{len(balance_data)} could not load financial statements for \t\t {stock}")

            error_load = pd.concat([error_load, pd.Series(stock)])

            x += 1 
            time.sleep(60)
            continue # skips rest of code in this loop 


        earnings_date = inc_data['fiscalDateEnding']
        currency = inc_data['reportedCurrency']

        first_date = datetime.datetime.strptime(earnings_date, '%Y-%m-%d')
        weekday = int(first_date.strftime('%w')) # Sunday is 0 and Saturday is 6. 

        # changes the date to the following friday if it is a weekday 
        if  weekday == 1: # Monday  
            first_date += datetime.timedelta(11)
        elif weekday == 2: 
            first_date += datetime.timedelta(10)
        elif weekday == 3: 
            first_date += datetime.timedelta(9)
        elif weekday == 4: 
            first_date += datetime.timedelta(8)
        elif weekday == 5: 
            first_date += datetime.timedelta(7) 
        elif weekday == 6: 
            first_date += datetime.timedelta(6)
        elif weekday == 0: # Sunday 
            first_date += datetime.timedelta(5)

        # has to be a multiple of 7 (Ex. 7 * 48 = 336)
        # second_date should be before next years annual earnings announcement
        second_date = first_date + datetime.timedelta(336)

        first_date = first_date.strftime('%Y-%m-%d')
        second_date = second_date.strftime('%Y-%m-%d')

        try: 
            first_price = price_data['Weekly Adjusted Time Series'][first_date]['5. adjusted close']
            second_price = price_data['Weekly Adjusted Time Series'][second_date]['5. adjusted close']
        except KeyError: 
            # len(price_data) should return 2, should return 0 if a valid api call was not performed, should return 1 if api call limit has been reache
            print(f"Error: {len(price_data)} price_data does not contain {first_date} or {second_date} \t\t ({stock})")

            error_date = pd.concat([error_date, pd.Series(stock)])
            x += 1 
            time.sleep(60)
            continue

        first_price = float(first_price)
        second_price = float(second_price)

        perc_change = (second_price - first_price) / first_price

        df_details = pd.DataFrame({'ticker': stock, 
                                   'sector': sector, 
                                   'industry': industry, 
                                   'earnings_date': earnings_date, 
                                   'currency': currency, 
                                   'current_price': first_price, 
                                   'following_price': second_price,
                                   'first_price_date': first_date,
                                   'second_price_date': second_date, 
                                   'percent_change': perc_change}, 
                                  index=[0])

        df_inc = pd.DataFrame(inc_data, index=[0])
        df_inc.drop(['fiscalDateEnding', 'reportedCurrency'], axis=1, inplace=True)

        df_bal = pd.DataFrame(bal_data, index=[0])
        df_bal.drop(['fiscalDateEnding', 'reportedCurrency'], axis=1, inplace=True)

        df_cas = pd.DataFrame(cas_data, index=[0])
        df_cas.drop(['fiscalDateEnding', 'reportedCurrency', 'netIncome'], axis=1, inplace=True)

        df_all = pd.concat([df_details, df_inc, df_cas, df_bal], keys=['details', 'income', 'cash', 'balance'], axis=1)

        master_df = pd.concat([master_df, df_all], ignore_index=True)



        print(x, stock)
        x += 1 

        time.sleep(60)

0 AAPL


KeyboardInterrupt: 

In [6]:
master_df

details                                                                      \
   ticker                  sector                                    industry   
0    AAPL  Information Technology  Technology Hardware, Storage & Peripherals   

                                                                         \
  earnings_date currency current_price following_price first_price_date   
0    2018-09-30      USD       54.0175         52.1629       2018-10-05   

                                    ...             balance  \
  second_price_date percent_change  ... currentLongTermDebt   
0        2019-09-06      -0.034333  ...          8784000000   

                                                                         \
  longTermDebtNoncurrent shortLongTermDebtTotal otherCurrentLiabilities   
0            93735000000           220132000000             33327000000   

                                                                   \
  otherNonCurrentLiabilities totalShareholderEquity treasuryStock   
0                48914000000           107147000000          None   

                                                              
  retainedEarnings  commonStock commonStockSharesOutstanding  
0      70400000000  40201000000                   4754986000  

[1 rows x 96 columns]

In [ ]:
cash_data['Note']

In [ ]:
master_df

In [ ]:
%store error_load 
%store error_date 
%store master_df
%store x 

In [ ]:
master_df

In [ ]:
# replaces any None values which are interpreted as a string which can not be changed to floats and then changes respective columns to floats
master_df.iloc[:, 8:] = master_df.iloc[:, 8:].replace('None', np.nan).astype(float)

In [ ]:
# flattens column index 
master_df.columns = master_df.columns.get_level_values(1)

In [ ]:
master_df.to_csv('./financial_statements.csv', index=False)